In [ ]:
!pip install gwcloud-python bilby

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import gwcloud_python
plt.style.use('publication')

import glob
import numpy as np
import bilby

In [ ]:
from gwcloud_python import GWCloud

### user token -- sign into gwcloud.org.au/auth/api-token.  Token is unique to individuals and provides authentication e.g., to LVC data and cluster resources
gwc = GWCloud(token='51d309e3a82c58384bda96db0bfb36cbb33048c1bac83f5576c1ef080b9057c6') 

In [ ]:
# Download the merge_result.json files for the eccentric and non-eccentric GW190521 runs.
target = ['GW190521', 'GW190521_eccentric']
all_jobs = gwc.get_user_jobs()
for job in all_jobs:
    if job.name in target:
        print(job.name)
        file_list = gwcloud_python.file_reference.FileReferenceList()
        json_files = job.get_full_file_list().filter_list_by_path(directory='result', extension='json')
        for file in json_files:
            if 'merge' in str(file.path):
                file_list.append(file)
                break
        job.save_files_by_reference(file_list, '.')

for file in glob.glob('result/*.json'):
    print(file)
    # Create a result object using the json file to access the events parameters.
    posterior_result = bilby.core.result.read_in_result(filename=file)
    duration = 8.
    sampling_frequency = 1024.
    minimum_frequency = 11

    # Specify the output directory and the name of the simulation.
    outdir = file[:-18] + ' h(t) reconstruction'
    label = 'GWCloud demonstration'
    bilby.core.utils.setup_logger(outdir=outdir, label=label)

    # Set up a random seed for result reproducibility.  This is optional!
    np.random.seed(88170235)

    # We are going to inject a binary black hole waveform.  We first establish a
    # dictionary of parameters that includes all of the different waveform
    # parameters, including masses of the two black holes (mass_1, mass_2),
    # spins of both black holes (a, tilt, phi), etc.
    parameters = ['mass_1', 'mass_2', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12', 'phi_jl', 'luminosity_distance',
                  'theta_jn', 'psi', 'phase', 'geocent_time', 'ra', 'dec']
    injection_parameters = {}
    for parameter in parameters: 
        injection_parameters[parameter] = np.array(posterior_result.posterior[parameter].sample(1))[0]
        print(injection_parameters[parameter])
    
    
    # Fixed arguments passed into the source model
    waveform_arguments = dict(waveform_approximant='IMRPhenomPv2',
                              reference_frequency=50.,
                              minimum_frequency=minimum_frequency)

    # Create the waveform_generator using a LAL BinaryBlackHole source function
    waveform_generator = bilby.gw.WaveformGenerator(
        duration=duration, sampling_frequency=sampling_frequency,
        frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
        parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
        waveform_arguments=waveform_arguments)

    # Set up interferometers.  In this case we'll use two interferometers
    # (LIGO-Hanford (H1), LIGO-Livingston (L1). These default to their design
    # sensitivity
    ifo = bilby.gw.detector.InterferometerList(['H1'])
    ifo.set_strain_data_from_power_spectral_densities(
        sampling_frequency=sampling_frequency, duration=duration,
        start_time=injection_parameters['geocent_time'] - 3)
    ifo.inject_signal(waveform_generator=waveform_generator,
                       parameters=injection_parameters)


    plt.plot(waveform_generator.time_array,waveform_generator.time_domain_strain()['plus'])
    plt.xlabel('time [s]')
    plt.xlim(7.5, 10)
    plt.ylabel('strain')

legend_elements = [Line2D([0],[0], color='orange', lw='3', ls='-', label='non-eccentric'),
                   Line2D([0],[0], color='blue', lw='3', ls='-', label='eccentric'),
                    ]

plt.legend(handles=legend_elements, loc = 'upper left', bbox_to_anchor=(1,1), fontsize=16)
plt.tight_layout()

plt.savefig('GW190521 ht.pdf')